In [31]:
import numpy as np
import pandas as pd

In [32]:
np.random.seed(42)

# === Оси времени ===
years = [2020, 2021, 2022, 2023, 2024]
months = ["Январ","Феврал","Март","Апрел","Май","Июн",
          "Июл","Август","Сентябр","Октябр","Ноябр","Декабр"]

# === Сезонные сдвиги климата ===
month_temp_shift = {"Январ":-10,"Феврал":-8,"Март":-4,"Апрел":2,"Май":8,
                    "Июн":12,"Июл":14,"Август":12,"Сентябр":8,
                    "Октябр":2,"Ноябр":-4,"Декабр":-8}
month_rain_shift = {"Январ":30,"Феврал":35,"Март":60,"Апрел":80,"Май":70,
                    "Июн":40,"Июл":30,"Август":25,"Сентябр":40,
                    "Октябр":50,"Ноябр":45,"Декабр":35}

# === Регионы ===
regions = {
    "Хатлон": {"zone":"Водии","temp":(18,28)},
    "Суғд": {"zone":"Нисбатан кӯҳӣ","temp":(14,24)},
    "ВМКБ": {"zone":"Кӯҳӣ","temp":(0,9)},
    "НТҶ": {"zone":"Мухталиф","temp":(16,26)}
}

In [33]:
# === Доли по маҳсулотҳо ===
shares = {
    "Гандум":   {"Хатлон":0.40,"Суғд":0.35,"НТҶ":0.20,"ВМКБ":0.05},
    "Сабзавот": {"Хатлон":0.45,"Суғд":0.30,"НТҶ":0.20,"ВМКБ":0.05},
    "Картошка": {"Хатлон":0.25,"Суғд":0.40,"НТҶ":0.25,"ВМКБ":0.10},
    "Гӯшт":     {"Хатлон":0.38,"Суғд":0.32,"НТҶ":0.20,"ВМКБ":0.10},
    "Шир":      {"Хатлон":0.40,"Суғд":0.30,"НТҶ":0.20,"ВМКБ":0.10},
}

# === Базовые страновые ориентиры ===
base_country_targets = {
    "Гандум":   {"area": 230_000, "yield": 3.0},
    "Картошка": {"area": 50_000,  "yield": 20.5},
    "Сабзавот": {"area": 85_000,  "yield": 15.5},
    "Гӯшт":     {"volume": 360_000},
    "Шир":      {"volume": 1_150_000},
}

In [34]:
# === Функция естественной динамики (±10% по годам) ===
def evolve_value(prev_val):
    change = np.random.uniform(-0.1, 0.1)
    return prev_val * (1 + change)

In [35]:
# === Генерация показателей для каждого года ===
country_targets_by_year = {}
prev = base_country_targets
for y in years:
    country_targets_by_year[y] = {}
    for k, v in prev.items():
        if "area" in v:
            area = evolve_value(v["area"])
            yld = evolve_value(v["yield"])
            country_targets_by_year[y][k] = {"area": area, "yield": yld}
        else:
            vol = evolve_value(v["volume"])
            country_targets_by_year[y][k] = {"volume": vol}
    prev = country_targets_by_year[y]

In [36]:
# === Урожайные месяцы ===
harvest_months = {
    "Гандум": ["Июн","Июл"],
    "Картошка": ["Май","Июн","Сентябр"],
    "Сабзавот": ["Июн","Июл","Август","Сентябр"],
    "Гӯшт": months,
    "Шир": months
}

In [37]:
# === Базовые цены (сом/т) ===
base_prices = {
    "Гандум": (5000, 6000),
    "Картошка": (6000, 8000),
    "Сабзавот": (4000, 6000),
    "Гӯшт": (60000, 80000),
    "Шир": (8000, 10000),
}

In [38]:
# === Чувствительность цены к дефициту ===
price_alpha = {
    "Гандум": 0.25,
    "Картошка":0.20,
    "Сабзавот":0.20,
    "Гӯшт":0.15,
    "Шир":0.15,
}

In [39]:
# === Урожайность: фиксированные по регионам ===
yield_mult_fixed = {
    "Хатлон": 1.20,
    "Суғд": 1.05,
    "НТҶ": 0.75,
    "ВМКБ": 0.20
}

In [40]:
# === Торговля (импорт/экспорт bias) ===
trade_bias = {
    "Хатлон": {"imp": 0.5, "exp": 1.1},
    "Суғд": {"imp": 0.7, "exp": 1.0},
    "НТҶ": {"imp": 1.4, "exp": 0.5},
    "ВМКБ": {"imp": 1.1, "exp": 0.1}
}

In [41]:
# === Цены: региональный коэффициент ===
price_region_bias = {
    "Хатлон": 1.25,
    "Суғд": 1.10,
    "НТҶ": 0.49,
    "ВМКБ": 0.21
}

In [42]:
# === Базовый спрос (т/мес) и сезонный коэффициент ===
base_demand = {
    "Гандум": 30000,
    "Картошка":20000,
    "Сабзавот":25000,
    "Гӯшт":15000,
    "Шир":40000
}

season_factor = {
    "Январ": 0.9,"Феврал": 0.9,"Март": 1.0,"Апрел": 1.1,"Май": 1.2,
    "Июн": 1.3,"Июл": 1.3,"Август": 1.2,"Сентябр": 1.1,"Октябр": 1.0,
    "Ноябр": 0.95,"Декабр": 0.9
}

In [43]:
# === Региональные верхние границы самодостаточности ===
region_selfsuff_cap = {
    "Хатлон": 60.0,
    "Суғд": 57.0,
    "НТҶ": 25.0,
    "ВМКБ": 5.0
}

In [44]:
rows = []

# === Основной цикл ===
for year in years:
    ct = country_targets_by_year[year]

    wheat_area_country   = np.random.uniform(ct["Гандум"]["area"]*0.97, ct["Гандум"]["area"]*1.03)
    wheat_yield_country  = np.random.uniform(ct["Гандум"]["yield"]*0.97, ct["Гандум"]["yield"]*1.03)
    veg_area_country     = np.random.uniform(ct["Сабзавот"]["area"]*0.97, ct["Сабзавот"]["area"]*1.03)
    veg_yield_country    = np.random.uniform(ct["Сабзавот"]["yield"]*0.97, ct["Сабзавот"]["yield"]*1.03)
    pot_area_country     = np.random.uniform(ct["Картошка"]["area"]*0.97, ct["Картошка"]["area"]*1.03)
    pot_yield_country    = np.random.uniform(ct["Картошка"]["yield"]*0.97, ct["Картошка"]["yield"]*1.03)
    meat_country         = np.random.uniform(ct["Гӯшт"]["volume"]*0.97, ct["Гӯшт"]["volume"]*1.03)
    milk_country         = np.random.uniform(ct["Шир"]["volume"]*0.97, ct["Шир"]["volume"]*1.03)

    def with_noise(x): return x * np.random.uniform(0.95, 1.05)

    area_by_region = {
        "Гандум":   {r: with_noise(wheat_area_country * shares["Гандум"][r]) for r in regions},
        "Сабзавот": {r: with_noise(veg_area_country * shares["Сабзавот"][r]) for r in regions},
        "Картошка": {r: with_noise(pot_area_country * shares["Картошка"][r]) for r in regions},
    }

    vol_by_region = {
        "Гӯшт": {r: with_noise(meat_country * shares["Гӯшт"][r]) for r in regions},
        "Шир":  {r: with_noise(milk_country * shares["Шир"][r]) for r in regions},
    }

    for region, rinfo in regions.items():
        # === Растанипарварӣ ===
        for product in ["Гандум","Картошка","Сабзавот"]:
            annual_area = area_by_region[product][region]
            months_h = harvest_months[product]
            n_harvest = max(1, len(months_h))
            for month in months:
                is_harvest = month in months_h
                area = (annual_area / n_harvest) if is_harvest else 0.0
                base_yield_reg = {
                    "Гандум": wheat_yield_country,
                    "Картошка": pot_yield_country,
                    "Сабзавот": veg_yield_country
                }[product] * yield_mult_fixed[region]

                yield_month = base_yield_reg if is_harvest else 0.0
                production = area * yield_month

                consumption = base_demand[product] * shares[product][region] \
                              * season_factor[month] * np.random.uniform(0.9, 1.1)

                # === Баланс и торг. ===
                net_need = consumption - production
                if net_need > 0:
                    imp = net_need * np.random.uniform(0.6, 0.95) * trade_bias[region]["imp"]
                    exp = 0.0
                else:
                    imp = 0.0
                    exp = (-net_need) * np.random.uniform(0.3, 0.9) * trade_bias[region]["exp"]

                eps = 1e-6
                self_suff = ((production + imp) / (consumption + eps)) * 100
                self_suff = min(region_selfsuff_cap[region], self_suff)

                base_p = np.random.uniform(*base_prices[product])
                if production <= 0.0:
                    price = 0.0
                else:
                    if is_harvest:
                        d_s_ratio = consumption / (production + imp + eps)
                        price = (base_p * (1 + price_alpha[product] * (d_s_ratio - 1))
                                 + np.random.normal(0, 0.25)) * price_region_bias[region]
                    else:
                        price = (base_p + np.random.normal(0, 0.25)) * price_region_bias[region]

                price = max(0.0, price)

                temp = np.random.uniform(*rinfo["temp"]) + month_temp_shift[month] + np.random.normal(0, 1.2)
                rain = month_rain_shift[month] + np.random.normal(0, 8)

                rows.append([
                    year, month, region, product,
                    round(area,1), round(yield_month,2), round(production,1),
                    round(price,2), round(imp,1), round(exp,1),
                    round(consumption,1), round(self_suff,1),
                    round(temp,1), round(rain,1), rinfo["zone"]
                ])

        # === Чорводори ===
        for product in ["Гӯшт","Шир"]:
            annual_prod = vol_by_region[product][region]
            for month in months:
                production = annual_prod / 12.0
                consumption = base_demand[product] * shares[product][region] \
                              * season_factor[month] * np.random.uniform(0.9, 1.1)

                net_need = consumption - production
                if net_need > 0:
                    imp = net_need * np.random.uniform(0.6, 0.95) * trade_bias[region]["imp"]
                    exp = 0.0
                else:
                    imp = 0.0
                    exp = (-net_need) * np.random.uniform(0.3, 0.9) * trade_bias[region]["exp"]

                eps = 1e-6
                self_suff = ((production + imp) / (consumption + eps)) * 100
                self_suff = min(region_selfsuff_cap[region], self_suff)

                base_p = np.random.uniform(*base_prices[product])
                d_s_ratio = consumption / (production + imp + eps)
                price = (base_p * (1 + price_alpha[product] * (d_s_ratio - 1))
                         + np.random.normal(0, 0.35)) * price_region_bias[region]
                price = max(0.0, price)

                temp = np.random.uniform(*rinfo["temp"]) + month_temp_shift[month] + np.random.normal(0, 1.2)
                rain = month_rain_shift[month] + np.random.normal(0, 8)

                rows.append([
                    year, month, region, product,
                    0.0, 0.0, round(production,1),
                    round(price,2), round(imp,1), round(exp,1),
                    round(consumption,1), round(self_suff,1),
                    round(temp,1), round(rain,1), rinfo["zone"]
                ])

df = pd.DataFrame(rows, columns=[
    "Сол","Моҳ","Минтақа","Маҳсулот",
    "Ҳаҷми замин (га)","Ҳосилнокӣ (т/га)","Истеҳсолот (т)",
    "Арзиш (сом/т)","Воридот (т)","Содирот (т)",
    "Истеъмолот (т)","Худтаъминкуни (%)",
    "Ҳарорат (°C)","Боришот (мм)","Минтақаи иқлимӣ"
])

df["Худтаъминкуни"] = np.where(df["Худтаъминкуни (%)"] >= 59, "Бале", "Не")

if __name__ == '__main__':
    display(df.head(10))

,Сол,Моҳ,Минтақа,Маҳсулот,Ҳаҷми замин (га),Ҳосилнокӣ (т/га),Истеҳсолот (т),Арзиш (сом/т),Воридот (т),Содирот (т),Истеъмолот (т),Худтаъминкуни (%),Ҳарорат (°C),Боришот (мм),Минтақаи иқлимӣ,Худтаъминкуни
0,2020,Январ,Хатлон,Гандум,0.0,0.00,0.0,0.00,4670.8,0.0,9881.0,47.3,11.2,37.5,Водии,Не
1,2020,Феврал,Хатлон,Гандум,0.0,0.00,0.0,0.00,3563.3,0.0,11385.9,31.3,11.6,42.8,Водии,Не
2,2020,Март,Хатлон,Гандум,0.0,0.00,0.0,0.00,3607.2,0.0,11594.2,31.1,18.2,70.8,Водии,Не
3,2020,Апрел,Хатлон,Гандум,0.0,0.00,0.0,0.00,5442.0,0.0,14222.2,38.3,27.6,92.3,Водии,Не
4,2020,Май,Хатлон,Гандум,0.0,0.00,0.0,0.00,5865.9,0.0,15180.4,38.6,30.4,67.6,Водии,Не
5,2020,Июн,Хатлон,Гандум,44033.5,3.92,172760.1,5698.69,0.0,105000.2,15020.8,60.0,35.8,33.5,Водии,Бале
6,2020,Июл,Хатлон,Гандум,44033.5,3.92,172760.1,5453.79,0.0,134528.2,16940.7,60.0,40.1,34.1,Водии,Бале
7,2020,Август,Хатлон,Гандум,0.0,0.00,0.0,0.00,6125.0,0.0,15530.6,39.4,38.1,22.4,Водии,Не
8,2020,Сентябр,Хатлон,Гандум,0.0,0.00,0.0,0.00,4677.5,0.0,12481.7,37.5,34.6,38.1,Водии,Не
9,2020,Октябр,Хатлон,Гандум,0.0,0.00,0.0,0.00,3999.3,0.0,11801.8,33.9,23.2,53.2,Водии,Не


In [ ]:
df.drop(columns='Боришот (мм)')

In [45]:
# === Сохранение ===
df.to_csv("tajikistan_agriculture_2020_2024_tj.csv", index=False, encoding="utf-8-sig")
print("✅ Синтетический датасет успешно создан и сохранён как 'tajikistan_agriculture_2023_2024_tj.csv'")

✅ Синтетический датасет успешно создан и сохранён как 'tajikistan_agriculture_2023_2024_tj.csv'
